In [5]:
import pyvisa as visa
from time import sleep
rm = visa.ResourceManager()
#odp = rm.open_resource('USB0::0x5345::0x1234::1909111::INSTR')
odp = rm.open_resource('USB0::0x1AB1::0x0E11::DP8B260200018::INSTR')
#print(0)
print(odp.query("*IDN?"))
#print(1)

RIGOL TECHNOLOGIES,DP832A,DP8B260200018,00.01.19



In [55]:
odp.write("INST:NSEL 1")
odp.write("VOLT:PROT 10.1")
odp.write("CURR:PROT 0.001")
odp.write("OUTP ON")

odp.write("INST:NSEL 2")
odp.write("VOLT:PROT 10.1")
odp.write("CURR:PROT 0.001")
odp.write("OUTP ON")

odp.write("INST:NSEL 3")
odp.write("VOLT:PROT 5")
odp.write("CURR:PROT 0.001")
odp.write("OUTP ON")

9

In [83]:
ON = "Turn on"
OFF = "Turn off"

TODO = ON


sleeptime=0.5

V1 = 1000   # (anode) kV < 2100 2200に戻す
V2 = 1000   # (out) kV < 2000 2100に戻す
V3 = 100    # in 

Vstep0 = 5*50/5000
V1g0 = 10*V1/5000
V2g0 = 10*V2/2500
V3g0 = 10*V3/1250
#print(odp.query("MEAS:VOLT?"))

In [79]:
if TODO == ON:
    odp.write("INST:NSEL 1")
    vinit1 = float(odp.query("MEAS:VOLT?"))
    sleep(0.1)
    odp.write("INST:NSEL 2")
    vinit2 = float(odp.query("MEAS:VOLT?"))
    sleep(0.1)
    odp.write("INST:NSEL 3")
    vinit3 = float(odp.query("MEAS:VOLT?"))
    while 1:
        odp.write("INST:NSEL 1")
        vcurr1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.2)
        odp.write("INST:NSEL 2")
        vcurr2 = float(odp.query("MEAS:VOLT?"))
        sleep(0.2)
        odp.write("INST:NSEL 3")
        vcurr3 = float(odp.query("MEAS:VOLT?"))
        sleep(0.2)
        if vcurr1 > V1g0+0.1:
            if vcurr2 > V2g0+0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    sleep(sleeptime)
            elif vcurr2 < V2g0-0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    sleep(sleeptime)
            else:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    sleep(sleeptime)
        elif vcurr1 < V1g0-0.1:
            if vcurr2 > V2g0+0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    sleep(sleeptime)
            elif vcurr2 < V2g0-0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    sleep(sleeptime)
            else:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1+Vstep0))
                    sleep(sleeptime)       
        else:
             if vcurr2 > V2g0+0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    sleep(sleeptime)
             elif vcurr2 < V2g0-0.1:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2+Vstep0))
                    sleep(sleeptime)
             else:
                if vcurr3 > V3g0+0.1:
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                elif vcurr3 < V3g0-0.1:
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3+Vstep0))
                    sleep(sleeptime)
                else:
                    break
        print(vcurr1, vcurr2, vcurr3)
    odp.write("INST:NSEL 1")
    odp.write("VOLT "+str(V1g0))
    odp.write("INST:NSEL 2")
    odp.write("VOLT "+str(V2g0))
    odp.write("INST:NSEL 3")
    odp.write("VOLT "+str(V3g0))
    print("Finish")
    print(vcurr1, vcurr2, vcurr3)

elif TODO == OFF:
    odp.write("INST:NSEL 1")
    vinit1 = float(odp.query("MEAS:VOLT?"))
    sleep(0.1)
    odp.write("INST:NSEL 2")
    vinit2 = float(odp.query("MEAS:VOLT?"))
    sleep(0.1)
    odp.write("INST:NSEL 3")
    vinit3 = float(odp.query("MEAS:VOLT?"))
    while 1:
        odp.write("INST:NSEL 1")
        vcurr1 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        odp.write("INST:NSEL 2")
        vcurr2 = float(odp.query("MEAS:VOLT?"))
        odp.write("INST:NSEL 3")
        vcurr3 = float(odp.query("MEAS:VOLT?"))
        sleep(0.1)
        if vcurr1 > 0.11:
            if vcurr2 > 0.11:
                if vcurr3 > 0.11:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    sleep(sleeptime)                    
            else:
                if vcurr3 > 0.11:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 1")
                    odp.write("VOLT "+str(vcurr1-Vstep0))
                    sleep(sleeptime)                    
                
        else:
            if vcurr2 > 0.11:
                if vcurr3 > 0.11:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                else:
                    odp.write("INST:NSEL 2")
                    odp.write("VOLT "+str(vcurr2-Vstep0))
                    sleep(sleeptime)                    
            else:
                if vcurr3 > 0.11:
                    odp.write("INST:NSEL 3")
                    odp.write("VOLT "+str(vcurr3-Vstep0))
                    sleep(sleeptime)
                else:
                    break
                
        print(vcurr1, vcurr2, vcurr3)
    odp.write("INST:NSEL 1")
    odp.write("VOLT 0.01")
    odp.write("INST:NSEL 2")
    odp.write("VOLT 0.01")                    
    odp.write("INST:NSEL 3")
    odp.write("VOLT 0.01")                    
    
    
else:
    print("Check your spelling.")

5.418 8.008 0.8015
5.418 8.008 0.8015
5.3861 7.9658 0.7536
5.354 7.9241 0.7536
5.3221 7.924 0.7058
5.2901 7.8819 0.6578
5.2901 7.8819 0.6578
5.2581 7.8397 0.6099
5.2262 7.8398 0.5618
5.1944 7.7982 0.514
5.1944 7.7982 0.514
5.1625 7.7558 0.4658
5.1317 7.7559 0.4174
5.1007 7.714 0.3689
5.1007 7.714 0.3689
5.0697 7.6721 0.3211
5.039 7.6566 0.2733
5.039 7.6566 0.2733
5.0083 7.6152 0.2254
4.9777 7.573 0.2052
4.9777 7.573 0.2052
4.9472 7.5315 0.1572
4.9167 7.4905 0.109
4.886 7.4497 0.109
4.8548 7.4092 0.109
4.8236 7.3903 0.109
4.7924 7.3682 0.109
4.7603 7.3274 0.109
4.7286 7.2861 0.1091
4.6976 7.2461 0.1091
4.6666 7.2457 0.109
4.6358 7.2062 0.109
4.6043 7.1668 0.109
4.573 7.1277 0.109
4.5419 7.0942 0.109
4.511 7.0887 0.109
4.4799 7.0496 0.109
4.4489 7.0106 0.109
4.4177 6.9716 0.109
4.3867 6.963 0.109
4.3556 6.9329 0.109
4.3248 6.8942 0.109
4.2935 6.856 0.109
4.2627 6.8184 0.109
4.2316 6.8182 0.109
4.2008 6.7807 0.1088
4.1696 6.744 0.109
4.1388 6.7071 0.109
4.1081 6.69 0.109
4.0774 6.6706 0.1

In [61]:
odp.query("MEAS:VOLT?")

'0.0016\n'

In [63]:
odp.write("INST:NSEL 1")
odp.write("VOLT 1")

8